# АЛГОРИТМ ПОСТРОЕНИЯ СЛУЧАЙНОГО ЛЕСА, СОСТОЯЩЕГО ИЗ ДЕРЕВЬЕВ

Для каждого:

сгенерировать выборку с помощью бутстрэпа;
построить решающее дерево по выборке: по заданному критерию мы выбираем лучший признак, делаем разбиение в дереве по нему и так до исчерпания выборки → дерево строится, пока в каждом листе не более объектов или пока не достигнем определенной высоты дерева → при каждом разбиении сначала выбирается несколько случайных признаков из исходных, и оптимальное разделение выборки ищется только среди них.
Таким образом, случайный лес — это бэггинг над решающими деревьями, при обучении которых для каждого разбиения признаки выбираются из некоторого случайного подмножества признаков.

## РЕАЛИЗАЦИЯ НА PYTHON  И ПОДБОР ПАРАМЕТРОВ
____
Теперь обучим случайный лес на простых данных и посмотрим, как можно подбирать параметры случайного леса для достижения наилучшего качества модели.

Потренируемся на данных, по которым мы будем предсказывать погоду. Датасет можно найти [здесь](https://lms.skillfactory.ru/assets/courseware/v1/ecd07340ed6b98d6fb556b3429156eb4/asset-v1:Skillfactory+DST-PRO+15APR2020+type@asset+block/temps_extended.csv).

Откроем его, удалим признаки, не относящиеся к предсказанию (от дня недели, например, или от года погода не зависит), разделим на тестовую и обучающуюся выборки:

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
weather = pd.read_csv('/content/drive/MyDrive/all_data/temps_extended.csv')

In [5]:
y = weather['actual']
X = weather.drop(['actual','weekday','month','day','year'],axis =1)
X_train, X_val, Y_train, Y_val=train_test_split(X,y,test_size=0.3, random_state=42)

Попробуем подобрать гиперпараметры таким образом, чтобы получить оптимальный результат.

Если мы запускаем случайный лес без настройки параметров, то по умолчанию они следующие:

In [6]:
from sklearn.ensemble import RandomForestRegressor
from pprint import pprint
rf = RandomForestRegressor(random_state = 42)
# Look at parameters used by our current forest
print('Параметры по умолчанию:\n')
pprint(rf.get_params())

Параметры по умолчанию:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


Попробуем подбирать разные значения для некоторых параметров. Для перебора вариантов возьмем следующие параметры:

* n_estimators 
* max_features 
* max_depth 
* min_samples_split 
* min_samples_leaf
* bootstrap

Мы можем сами указать, какие значения гиперпараметров надо перебрать.

Зададим сетку гиперпараметров, которые будут перебираться:

In [9]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [10]:
# Обучим наш лес:
rf = RandomForestRegressor(random_state=42)
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid, n_iter=100, 
                               cv=3, verbose=2, random_state=42, n_jobs=-1)
rf_random.fit(X_train, Y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  6.9min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 12.8min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                              

In [11]:
# посмотрим, какие гиперпараметры нам предлагают как оптимальные:
rf_random.best_params_

{'bootstrap': True,
 'max_depth': 10,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 5,
 'n_estimators': 1000}

Обучите случайный лес с предустановленными параметрами и теми параметрами, которые мы отобрали как оптимальные. В обоих вариантах поставьте random_state =42. Какое улучшение MSE дала подстановка отобранных гиперпараметров? Ответ округлите до одного знака после запятой.

In [12]:
from sklearn.metrics import mean_squared_error

In [13]:
rf.fit(X_train, Y_train)
y_pred = rf.predict(X_val)
mean_squared_error(Y_val, y_pred)

24.570648328267477

In [16]:
rf = RandomForestRegressor(random_state=42, max_depth=10, max_features='sqrt', min_samples_leaf=2, min_samples_split=5, n_estimators=1000)
rf.fit(X_train, Y_train)
y_pred = rf.predict(X_val)
mean_squared_error(Y_val, y_pred)

23.10053911088241

In [17]:
24.570648328267477 - 23.10053911088241

1.470109217385069